# IMPORTS

In [1]:
import pandas as pd
import numpy as np
import os
from decimal import Decimal
import pickle
from pathlib import Path

# SETUP

In [2]:
dir_tree_util_path = os.path.join("utils", "dir_tree.py")
exec(open(dir_tree_util_path).read())

# INPUTS

In [3]:
# Paths
path_input = PROJECT_DIRS["DADOS_VEQ_ANTT_DIR"]
path_output = PROJECT_DIRS["DADOS_DERIVADOS_DIR"]

In [4]:
periodo = list(range(2010, 2025))

In [5]:
periodos_CAGR = [[2010,2023],[2016,2023]]

# Criando um dataframe consolidado com os dados de tráfego

In [6]:
# arquivos dos dados
arquivos = os.listdir(path_input)

In [7]:
df_trafego = pd.DataFrame()
for ano in periodo:
    arquivo = [a for a in arquivos if a[:-4].endswith(str(ano))][0]
    df_ano = pd.read_csv(os.path.join(path_input, arquivo), sep=';', encoding='cp1252', low_memory=False)
    df_trafego = pd.concat([df_trafego,df_ano])

# EDA Inicial

In [8]:
df_trafego.head()

,concessionaria,mes_ano,sentido,praca,categoria,tipo_de_veiculo,volume_total,multiplicador_de_tarifa,volume_veiculo_equivalente,tipo_de_cobranca
0,RODOVIA DO AÇO,01-01-2010,Decrescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,44146,1,44146,NaN
1,RODOVIA DO AÇO,01-01-2010,Crescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,35771,1,35771,NaN
2,RODOVIA DO AÇO,01-02-2010,Decrescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,33455,1,33455,NaN
3,RODOVIA DO AÇO,01-02-2010,Crescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,27109,1,27109,NaN
4,RODOVIA DO AÇO,01-03-2010,Decrescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,31677,1,31677,NaN


In [9]:
df_trafego.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000353 entries, 0 to 2329761
Data columns (total 10 columns):
 #   Column                      Dtype 
---  ------                      ----- 
 0   concessionaria              object
 1   mes_ano                     object
 2   sentido                     object
 3   praca                       object
 4   categoria                   object
 5   tipo_de_veiculo             object
 6   volume_total                object
 7   multiplicador_de_tarifa     object
 8   volume_veiculo_equivalente  object
 9   tipo_de_cobranca            object
dtypes: object(10)
memory usage: 251.8+ MB


In [10]:
for col in df_trafego:
    print(f"{col}: {df_trafego[col].isna().sum()}")

concessionaria: 0
mes_ano: 0
sentido: 0
praca: 0
categoria: 0
tipo_de_veiculo: 0
volume_total: 344
multiplicador_de_tarifa: 0
volume_veiculo_equivalente: 0
tipo_de_cobranca: 74780


In [11]:
df_trafego[df_trafego['volume_total'].isna()].sample(9)

,concessionaria,mes_ano,sentido,praca,categoria,tipo_de_veiculo,volume_total,multiplicador_de_tarifa,volume_veiculo_equivalente,tipo_de_cobranca
135297,RIOSP,01/05/2023,Crescente,"Praça 08 BR-116/SP km 205,00",Categoria 3,Passeio,NaN,"1,50","0,00",N/I
125202,ECOSUL,01/03/2023,Crescente,"Praça 01 BR-116/RS km 430,79",Categoria 9,Moto,NaN,"0,50","0,00",N/I
125352,ECOSUL,01/04/2023,Decrescente,"Praça 03 BR-116/RS km 541,20",Categoria 9,Moto,NaN,"0,50","0,00",N/I
135766,RIOSP,01/07/2023,Crescente,"Praça 08 BR-116/SP km 205,00",Categoria 5,Passeio,NaN,"2,00","0,00",N/I
126092,ECOSUL,01/11/2023,Decrescente,"Praça 05 BR-392/RS km 111,47",Categoria 9,Moto,NaN,"0,50","0,00",N/I
130086,ECOVIAS DO CERRADO,01/06/2023,Decrescente,"P7 - Jataí BR 364/GO km 156,550",Veículo Comercial Acima 10 eixos,Comercial,NaN,"10,00","0,00",N/I
125972,ECOSUL,01/10/2023,Decrescente,"Praça 04 BR-392/RS km 52,30",Categoria 9,Moto,NaN,"0,50","0,00",N/I
135055,RIOSP,01/04/2023,Crescente,"Praça 08 BR-116/SP km 205,00",Categoria 7,Comercial,NaN,"5,00","0,00",N/I
125292,ECOSUL,01/03/2023,Decrescente,"Praça 05 BR-392/RS km 111,47",Categoria 9,Moto,NaN,"0,50","0,00",N/I


In [12]:
df_trafego[df_trafego['volume_total'].isna()]['volume_veiculo_equivalente'].unique()

array(['0,00'], dtype=object)

In [13]:
df_trafego['mes_ano'].str[:2].unique()

array(['01', '14', '28', '31', '30', '02', '03', '04', '05', '06', '07',
       '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19',
       '20', '21', '22', '23', '24', '25', '26', '27', '29'], dtype=object)

In [14]:
df_trafego['mes_ano'].str[3:5].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12'], dtype=object)

In [15]:
df_trafego['tipo_de_cobranca'].unique()

array([nan, 'N/I', 'Manual', 'Automática', 'Mista'], dtype=object)

In [16]:
df_trafego['volume_total'].isna().any()

np.True_

In [17]:
df_trafego['volume_total'].isna().sum()

np.int64(344)

In [18]:
df_trafego['multiplicador_de_tarifa'].unique()

array(['1', '2', '1,5', '3', '4', '5', '6', '0,5', '7', '8', '9', '10',
       1.0, 2.0, 1.5, 3.0, 4.0, 5.0, 6.0, 0.5, 7.0, 8.0, 9.0, 10.0,
       '2,00', '3,00', '4,00', '5,00', '6,00', '0,50', '1,00', '1,50',
       '7,00', '8,00', '9,00', '10,00', '0,00', '11,00', '13,00', '15,00',
       '12,00', '14,00', '18,00', '20,00', '16,00', '17,00', '19,00'],
      dtype=object)

# Tratamento dos dados

In [19]:
# Colunas de ano, mes e dia
df_trafego['year'] = df_trafego['mes_ano'].str[-4:].astype('int') 
df_trafego['month'] = df_trafego['mes_ano'].str[3:5].astype('int')
df_trafego['day'] = df_trafego['mes_ano'].str[:2].astype('int') 

In [20]:
# Recriando a coluna de mes_ano, agora como datetime ao invés de string. Tem que ser em ingles pq a função do pandas exige
df_trafego['mes_ano'] = pd.to_datetime(df_trafego[['year', 'month', 'day']]).dt.date

In [21]:
# preenchendo os NaNs, qdo aplicável:
df_trafego['volume_total'] = df_trafego['volume_total'].fillna(0) #o volume equivalente é zero sempre que volume_total é zero

In [22]:
# Convertendo os tipos de dados para numéricos:
df_trafego['volume_total'] = df_trafego['volume_total'].apply(lambda x: x.replace(',','.') if type(x) == str else x)
df_trafego['volume_total'] = df_trafego['volume_total'].apply(lambda x: x.split('.')[0] if type(x) == str else x)
df_trafego['volume_total'] = df_trafego['volume_total'].astype('int')

df_trafego['volume_veiculo_equivalente'] = df_trafego['volume_veiculo_equivalente'].apply(lambda x: x.replace(',','.') if type(x) == str else x)
df_trafego['volume_veiculo_equivalente'] = df_trafego['volume_veiculo_equivalente'].astype('float') # existe multiplicador de tarifa fracionário

In [23]:
# convertendo categoria para string (do contrário não salva para parquet):
df_trafego['categoria'] = df_trafego['categoria'].astype('string')

In [24]:
# convertendo o multiplicador de tarifa de string para decimal (p/manter a precisao):
df_trafego['multiplicador_de_tarifa'] = df_trafego['multiplicador_de_tarifa'].apply(lambda x: x.replace(',','.') if type(x) == str else x)
df_trafego['multiplicador_de_tarifa'] = df_trafego['multiplicador_de_tarifa'].apply(Decimal)

In [25]:
# O tipo de tráfego ora está em maisúcula, ora em minúscula.
# Colocando tudo p/maiúscula
df_trafego['tipo_de_veiculo'] = df_trafego['tipo_de_veiculo'].str.upper()

In [26]:
# algumas concessionárias estão ora em maiúsculas ora em minúsculas
# colocando tudo para maiúscula
df_trafego['concessionaria'] = df_trafego['concessionaria'].str.upper()

In [27]:
df_trafego.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000353 entries, 0 to 2329761
Data columns (total 13 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   concessionaria              object 
 1   mes_ano                     object 
 2   sentido                     object 
 3   praca                       object 
 4   categoria                   string 
 5   tipo_de_veiculo             object 
 6   volume_total                int64  
 7   multiplicador_de_tarifa     object 
 8   volume_veiculo_equivalente  float64
 9   tipo_de_cobranca            object 
 10  year                        int64  
 11  month                       int64  
 12  day                         int64  
dtypes: float64(1), int64(4), object(7), string(1)
memory usage: 320.5+ MB


In [28]:
df_trafego['year'].unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       2021, 2022, 2023, 2024])

In [29]:
df_trafego.query("year == 2024")['month'].unique()

array([1, 2, 3, 4, 5, 6])

## Criando coluna desambiguando categorias comercial x passeio

In [30]:
df_trafego['tipo_de_veiculo'].unique()

array(['PASSEIO', 'COMERCIAL', 'MOTO', 'VEÍCULO PEQUENO'], dtype=object)

In [31]:
df_trafego["TIPO_TRAFEGO"] = df_trafego['tipo_de_veiculo'].apply(lambda x: x if x == "COMERCIAL" else "PASSEIO")

In [32]:
df_trafego["TIPO_TRAFEGO"].unique()

array(['PASSEIO', 'COMERCIAL'], dtype=object)

## Criando colunas com trafégo em Veqs para comercial e passeio

In [33]:
df_trafego = df_trafego.rename(columns={'volume_veiculo_equivalente':'VEQS_TOTAL'})
df_trafego['VEQS_COMERCIAL'] = np.where(
    df_trafego['TIPO_TRAFEGO'] == 'COMERCIAL',
    df_trafego['VEQS_TOTAL'],
    0
)
df_trafego['VEQS_PASSEIO'] = np.where(
    df_trafego['TIPO_TRAFEGO'] == 'PASSEIO',
    df_trafego['VEQS_TOTAL'],
    0
)

# Criando dataframe com o período inicial e final dos dados por concessionária

In [34]:
df_periodos = df_trafego.groupby('concessionaria').agg(
    data_inicial=('mes_ano', 'min'),
    data_final=('mes_ano', 'max')
).reset_index()
df_periodos['data_inicial'] = pd.to_datetime(df_periodos['data_inicial']).dt.date
df_periodos['data_final'] = pd.to_datetime(df_periodos['data_final']).dt.date

In [35]:
df_periodos.index = df_periodos['concessionaria']
df_periodos = df_periodos.drop(columns=['concessionaria'])

In [36]:
df_periodos = df_periodos.sort_values(by='data_inicial', ascending=False)

# Criando dataframes agrupando VEQs por ANO/concessionaria/tipo de trafego

In [47]:
df_trafego.columns

Index(['concessionaria', 'mes_ano', 'sentido', 'praca', 'categoria',
       'tipo_de_veiculo', 'volume_total', 'multiplicador_de_tarifa',
       'VEQS_TOTAL', 'tipo_de_cobranca', 'year', 'month', 'day',
       'TIPO_TRAFEGO', 'VEQS_COMERCIAL', 'VEQS_PASSEIO', 'MES_ANO_f'],
      dtype='object')

In [48]:
cols_veqs = ['VEQS_TOTAL','VEQS_COMERCIAL','VEQS_PASSEIO']

In [49]:
dict_veqs = {}
for col in cols_veqs:
    dict_veqs[col] = df_trafego.pivot_table(
        index='concessionaria',
        columns='year',
        values=col,
        aggfunc='sum'
        )

# Criando dataframes agrupando VEQs por MÊS/concessionaria/tipo de trafego

In [74]:
# Criando uma coluna agrupando todos os registros em um mes ano para o final do mes para groupby
df_trafego['MES_ANO_f'] = pd.to_datetime(df_trafego['mes_ano'], format='%m/%Y')
df_trafego['MES_ANO_f'] = df_trafego['MES_ANO_f'].dt.to_period('M').dt.to_timestamp('M').dt.date

In [75]:
dict_veqs_mensal = {}
for col in cols_veqs:
    dict_veqs_mensal[col] = df_trafego.pivot_table(
        index='concessionaria',
        columns='MES_ANO_f',
        values=col,
        aggfunc='sum'
        )

In [76]:
dict_veqs_mensal['VEQS_TOTAL']

MES_ANO_f,2010-01-31,2010-02-28,2010-03-31,2010-04-30,2010-05-31,2010-06-30,2010-07-31,2010-08-31,2010-09-30,2010-10-31,...,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30
concessionaria,,,,,,,,,,,,,,,,,,,,,
AUTOPISTA FERNÃO DIAS,8378489.0,7597815.5,8464339.0,8507557.5,9119625.5,8945909.5,9831941.0,9534627.5,11259964.0,12443127.0,...,14026920.00,14471788.50,14244332.00,14903341.00,12264573.0,NaN,NaN,NaN,NaN,NaN
AUTOPISTA FLUMINENSE,3624722.0,3291562.5,3352859.5,3112184.5,3368807.5,3241332.5,3533522.5,3543010.5,3529144.5,3617006.5,...,3846118.50,3971101.50,4001603.50,4235383.00,4195316.5,3780432.5,1451661.0,43079.5,NaN,NaN
AUTOPISTA LITORAL SUL,9715122.5,8449151.0,8635409.0,8207830.5,8408850.0,8159287.0,8963336.5,8979786.0,9152392.5,9477581.5,...,12418718.50,12311317.00,12892038.00,13552705.50,6103640.5,13317610.0,13921360.0,12593523.0,NaN,NaN
AUTOPISTA PLANALTO SUL,1990969.0,1967207.0,2199601.5,2154696.0,2267816.0,2108400.5,2214808.5,2197724.5,2147746.5,2221816.5,...,2549115.00,3047892.00,2833235.00,2775849.00,2477717.0,2958571.0,3142100.5,3187702.5,NaN,NaN
AUTOPISTA REGIS BITTENCOURT,10233460.5,9255794.5,11202293.0,10561292.5,11113305.0,10625262.0,11367074.5,11401061.5,11344991.5,11962673.0,...,13163838.00,13572890.00,13602677.50,13672725.50,13940858.5,7297.0,NaN,NaN,NaN,NaN
CONCEBRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8089644.50,8233267.50,7792289.00,7811751.00,7704918.5,7184002.5,7718145.5,7561248.5,7926717.5,7937696.5
CONCEPA,3152009.0,2866194.5,2321640.5,2213994.5,2152870.5,2053017.0,2212338.5,2222965.5,2356904.0,2499998.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CONCER,2139548.5,1843578.5,2017238.5,1959747.0,2016398.0,1979750.5,2203094.5,2137830.5,2102409.5,2196490.5,...,2046140.50,2148980.00,2114233.00,2255075.50,1769030.0,1905405.0,1978087.5,2006069.5,2083291.5,577255.0
CRO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10454277.50,9905309.50,9858427.50,9427826.00,7109570.0,6889198.0,7646984.5,8702620.0,9509593.0,720864.0


# Salvando p/parquet

In [55]:
# dataframe total
# df_trafego.to_parquet(os.path.join(path_dados_derivados,'df_trafego.parquet'))
df_trafego.to_parquet(path_output /'df_trafego.parquet')

In [56]:
# data inicial e final
df_periodos.to_parquet(path_output /'df_periodos.parquet')

In [63]:
# dicionário com os dados de tráfego anuais:
for veq in dict_veqs:
    dict_veqs[veq].to_parquet(path_output / f'df_{veq}_anual.parquet')

In [77]:
# dicionário com os dados de tráfego mensial:
for veq in dict_veqs_mensal:
    dict_veqs_mensal[veq].to_parquet(path_output / f'df_{veq}_mensal.parquet')

# Salvando p/Excel

In [61]:
# dados de tráfego anual
with pd.ExcelWriter(path_output / 'dados_trafego_ANTT_anual.xlsx') as writer:
    df_periodos.to_excel(writer, sheet_name="periodo_concessionarias")
    for veq in dict_veqs:
        dict_veqs[veq].to_excel(writer, sheet_name=f"{veq}")

In [73]:
# dados de tráfego mensal
with pd.ExcelWriter(path_output / 'dados_trafego_ANTT_mensal.xlsx') as writer:
    df_periodos.to_excel(writer, sheet_name="periodo_concessionarias")
    for veq in dict_veqs_mensal:
        dict_veqs_mensal[veq].to_excel(writer, sheet_name=f"{veq}")